# Lab 0:  Review and warm-up
Welcome to the first Data 102 lab! 

The goal of this lab is to review some basic probability and programming. We will also learn more about binary decision making.

The code (and answers) that you need to write is commented out with a message **"TODO: ..."**. There is additional documentation for each part as you go along.


## Collaboration Policy
Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others please **include their names** in the cell below.

## Submission
To submit this assignment, rerun the notebook from scratch (by selecting Kernel > Restart & Run all), and then print as a pdf (File > download as > pdf) and submit it to Gradescope.

**For full credit, this assignment should be completed and submitted before Tuesday, August 30, 2022 at 11:59 PM. PST**

## Collaborators
Write the names of your collaborators in this cell.

`<Collaborator Name> <Collaborator e-mail>`

# Setup
Let's begin by importing the libraries we will use. You can find the documentation for the libraries here:
* matplotlib: https://matplotlib.org/3.1.1/contents.html
* numpy: https://docs.scipy.org/doc/
* pandas: https://pandas.pydata.org/pandas-docs/stable/
* seaborn: https://seaborn.pydata.org/

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

import timeit
import hashlib
%matplotlib inline

sns.set(style="dark")
plt.style.use("ggplot")

def get_hash(num):  # <- helper function for assessing correctness
    return hashlib.md5(str(num).encode()).hexdigest()

# Question 0: Vectorized operations in NumPy

We'll start this lab by reviewing the concept of **vectorization**. Many of the functions in NumPy (and pandas) are optimized to be much faster than the equivalent code using a `for` loop. This is because NumPy uses optimized and pre-compiled code written in a low-level language (in this case, C) to carry out mathematical operations. By using NumPy's vectorized operations instead of iterating explicitly (e.g., writing `for` loops), we can make our code run much faster. In some cases, this difference is small, but you'll see in future labs and homework assignments that sometimes it has a big impact.

Let's see vectorization in action and measure the time it takes to perform some vectorized and non-vectorized tasks on NumPy arrays. We'll start by summing numbers from 0 to 14,999. 

In [ ]:
%%timeit

sum_nonvect = 0
for item in range(0, 15000):
     sum_nonvect += item

In [ ]:
%%timeit

sum_vect = np.sum(np.arange(15000))

### a) How much faster is the vectorized version? Your answer should be a multiplicative factor (e.g., "it takes half as long").

`TODO: YOUR ANSWER HERE`


Now, consider the following array:

In [ ]:
arr = np.arange(0, 60, 3).reshape(4, 5)
arr

Suppose we want to compute a new array where each entry is the average of two neighboring entries in the original array. So, the first row would look like `[1.5, 4.5, 7.5, 10.5]` (we'll see examples of operations like this later in the class). Let's try doing this two different ways:

In [ ]:
%%timeit
# Using nested for loops

new_array_slow = np.zeros([arr.shape[0], arr.shape[1] - 1])
for i in range(arr.shape[0]):
    for j in range (arr.shape[1] - 1):
        new_array_slow[i,j] = (arr[i,j] + arr[i, j+1])/2

In [ ]:
%%timeit

new_array_fast = (arr[:, :-1] + arr[:, 1:])/2

### b) Suppose your friend looks at this and says that while the vectorized version does run faster, both versions run fast enough that it doesn't matter. Give two reasons your friend might be wrong.

*Hint: What happens if an algorithm needs to run this kind of operation many times?*

*Hint: try replacing `arr` with `np.random.random([2000, 2000])`. How do the results change?*

`TODO: your answers here`

# Question 1: The sinking of the Titanic
On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg. Unfortunately, there weren’t enough lifeboats for everyone onboard, resulting in the death of 1502 out of 2224 passengers and crew. 

In this question we will work with data about passengers traveling on the Titanic, and we will try to understand whether some groups of people were more likely to survive than others. We will also fit a binary classifier to the data and check its performance (e.g. how many false positive does it produce?) 

Let's start by importing the Titanic dataset using Pandas.

In [ ]:
titanic = pd.read_csv("titanic.csv")
titanic.head()

Here is a brief description of the variables included in the dataframe:
* **Survived**: binary variable taking value 1 if the person survived the shipwreck, 0 otherwise;
* **Pclass**: whether the passenger was traveling in 1st, 2nd, or 3rd class; 
* **Name**: passenger's name;
* **Sex**: passenger's gender;
* **Age**: passenger's age;
* **Siblings/Spouses Aboard**: how many siblings or spouses the passenger is traveling with;
* **Parents/Children Aboard**: how many parents or children the passenger is traveling with;
* **Fare**: what was the fare that the passenger paid.

## Part 1.a: Dataset check
We will first check some general properties of the dataframe. For example, we will see how many rows and columns the dataframe has and if there are missing values.

In [ ]:
# TODO: find the number of rows and columns in the dataframe 

In [ ]:
# TODO: check how many missing values there are per column

Running the cell below, we can also check how many different values each variable takes.

In [ ]:
titanic.nunique()

## Part 1.b: Exploring the dataset
We'll now look into more detail at some descriptive statistics and plots. We would like to know what is the percentage of people who survived the shipwreck and what is the passengers' distribution in terms of sex, age, class, etc.

For a refresher on data visualization using pandas and seaborn, you can refer to [Chapter 10 of the Data 100 textbook](https://www.textbook.ds100.org/ch/11/viz_intro.html).

Let's start by computing the percentage of people in the dataset who survived the shipwreck.

In [ ]:
# TODO: compute the percentage of people who survived (expressed as a decimal between 0 and 1)
survival_prob = ...

In [ ]:
# Validation tests: do not modify!
assert(get_hash(survival_prob)== '4fe6b7a204127acbec520ac2997133dc')
print("Test passed!")

The cell below uses the `countplot` function from seaborn to generate a plot displaying how many passengers were traveling in each of the three classes.

In [ ]:
sns.countplot(x = titanic["Pclass"]);

We can see that the distribution is not uniform, and that the majority of the passengers were traveling in third class. 

In the next question, write a line of code to generate a bar graph for the distribution of passenger sex.

In [ ]:
# TODO: use countplot to show how the distribution of passenger sex

In [ ]:
# TODO: use countplot to show the distribution of the number of parents/children aboard

Summing up: there were nearly twice as many male passengers as female passengers, and the majority of people were traveling alone.

Let's conclude by looking at the distribution of age and fare paid. In the next cell, write a line of code to visualize the distribution of passenger ages. *Hint: since the age column contains numerical data, you shouldn't use `sns.countplot`, which is meant for categorical data. Is there another seaborn function you can use?*

In [ ]:
# TODO: visualize the distribution of passenger ages

You should see that most travelers were in their 20s and 30s, and there were also many babies. What is the age of the youngest passenger? And the oldest?

In [ ]:
min(titanic["Age"]) 

In [ ]:
# TODO: print the age of the oldest passenger 
max_age = ...

In [ ]:
# Validation tests: do not modify!
assert(get_hash(max_age) == '8ee5d21b272d43a875504f3e5845e141')
print("Test passed!")

In [ ]:
# TODO: use displot to show the fare distribution

The fare distribution is really skewed: most people spent very little, but there is someone who spent even more than 500! Who are these people?

In [ ]:
# TODO: display the rows corresponding to passengers who spent more than 500

## Part 1.c: Conditional probabilites

So far, we have looked at the variables separately, considering each marginal distribution. This tells us about them individually, but we're also interested in the relationships between variables. In particular, we want to find connections between survival and the other variables. For example, we might wonder whether people in first class had a higher probability of surviving than people in third class or if females survived more or less often than males. To answer this type of question, we'll need to look at **conditional probabilities**. For a refresher on conditional probabilities, you can refer to [Section 4.4 of the Data 140 textbook](http://prob140.org/textbook/content/Chapter_04/04_Conditional_Distributions.html).

Let's start by looking at the conditional probabilities of survival given class type. If we denote the survival variable by $S$ and the class variable by $C$, we are comparing:

$$\mathbb{P}(S = 1 | C = 1) \quad \text{ vs } \quad \mathbb{P}(S = 1 | C = 2) \quad \text{ vs }\quad \mathbb{P}(S = 1 | C = 3)$$

We'll first compute $\mathbb{P}(S = 1 | C = 1)$, the probability of surviving given that the passenger has a first-class ticket. To do so, remember the definition of conditional probability: for two events $A$ and $B$,

$$
P(A|B) = \frac{P(A \text{ and } B)}{P(B)}
$$

In this case, we don't have to use Bayes' rule since we can compute the conditional probably directly using the definition.

In [ ]:
# TODO: complete the following code to compute P(S=1|C=1)
firstclass_survival_prob = np.mean((titanic["Survived"] == ...) & (titanic["Pclass"] == ...)) / np.mean(titanic["Pclass"] == ...)
firstclass_survival_prob

In [ ]:
# Validation tests: do not modify!
assert get_hash(np.round(firstclass_survival_prob, 2)) == 'b9c48c2d04160ef1ff72dba569292058'
print('Test passed!')

So, almost 63% of passengers traveling in first class survived: not bad! 

What about people in the other two classes?

In [ ]:
# TODO: compute the conditional probability of survival given that the passenger is in class 2
secondclass_survival_prob = ...
secondclass_survival_prob

In [ ]:
# Validation tests: do not modify!
assert get_hash(np.round(secondclass_survival_prob, 2)) == '7a7763d4618eb5666515c04dd53a3c08'
print('Test passed!')

In [ ]:
# TODO: compute the conditional probability of survival given that the passenger is in class 3
thirdclass_survival_prob = ...
thirdclass_survival_prob

In [ ]:
# Validation tests: do not modify!
assert get_hash(np.round(thirdclass_survival_prob, 2)) == '5d6182b8169f820c3e247e91131138ea'
print('Test passed!')

Survival probabilities were very different for the three class types! High class passenger had higher chances to survive.

We can also visualize survivals by class running the following cell.

In [ ]:
sns.countplot(x = 'Pclass', hue = "Survived", data = titanic);

You can keep on exploring and play with the other variables too, checking if there are groups with high or low probability of surviving. Try to condition on more than one variable (e.g. condition on both class type and gender).

## Part 1.d: Binary classification

In this last part, we're going to use logistic regression to predict passenger survival from the other variables. Here are the steps we'll follow:

1. Split the dataset into a training and test set
2. Fit a logistic regression model to the training set
3. Evaluate the performance on the test set using the language of binary decision-making that you saw in last week's lecture

We've done most of steps 1 and 2 for you, but we **strongly** encourage you to read through and understand all the code, since you'll need to do it yourself in future labs and homework assignments. Some resources that might be helpful:

* [Data 100 textbook: logistic regression](https://www.textbook.ds100.org/ch/19/classification_log_reg.html)
* [Data 100 textbook: one-hot encoding](https://www.textbook.ds100.org/ch/15/linear_feature_eng.html)
* scikit-learn's documentation for `LabelEncoder` and `train_test_split` (you'll have to find these on your own).


Running the following cell will prepare the data and split it into training and test sets.

In [ ]:
titanic["Sex"] = LabelEncoder().fit_transform(titanic["Sex"]) # encode gender variable (from string to integer)
titanic = titanic.drop("Name", axis = 1) # drop Name column

X = titanic.drop("Survived", axis = 1) # independent variables
y = titanic["Survived"] # dependent variable

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 3)

Now, we are ready to fit our logistic regression classifier and to predict which passengers in the test data survived.

In [ ]:
lr = LogisticRegression(random_state = 0)
lr.fit(X_train, y_train) # fit the classifier to the training data
yhat = lr.predict(X_test) # predict survival for test data

Complete the code in the following cell to generate a new dataframe with two columns: the first containing our **decisions** and the second containing the values in **reality**.

In [ ]:
output = pd.DataFrame()
output["Decisions"] = ...
output["Reality"] = y_test.values
output

In [ ]:
# Validation test: do not modify
assert get_hash(output['Decisions'].sum()) == 'e2ef524fbf3d9fe611d5a8e90fefdc9c'
assert get_hash((output['Decisions'] & output['Reality']).sum()) == 'ad61ab143223efbc24c7d2583be69251'
print('Test passed!')

We're now interested in evaluating the performance of the logistic regression classifier: let's check how many false positive, false negative, true positive and true negative we have obtained.

In [ ]:
# TODO: compute the number of false positives (FP), false negatives (FN), true positives (TP), and true negatives (TN)
FP = ...
FN = ...
TP = ...
TN = ...

In [ ]:
# Validation test: do not modify
performance = [FP, FN, TP, TN]
hash_list = ['37693cfc748049e45d87b8c7d8b9aacd', 
             '6364d3f0f495b6ab9dcf8d3b5c6e0b01', 
             'ad61ab143223efbc24c7d2583be69251', 
             '013d407166ec4fa56eb1e1f8cbe183b9']
assert all([get_hash(num) == hash_list[i] for (i, num) in enumerate(performance)])
print("Test passed!!!")

Using the four numbers you found above, answer the two multiple choice questions below. *Hint: It might help to draw out the 2-by-2 table as in lecture.*

In which of the following two settings did the classifier perform better?

(a) People who actually survived

(b) People who actually did not survive


In which of the following two settings did the classifier perform better?

(a) People for whom the classifier predicted they survived

(b) People for whom the classifier predicted they did not survive

`TO DO: your answers here.`

## Final tests
If all the tests below pass, you can assume you have successfuly completed the testable parts of the lab. Don't worry about understanding the code below; just make sure no asserts fail.

In [ ]:
tests = [survival_prob, max_age, np.round(firstclass_survival_prob, 2), 
         np.round(secondclass_survival_prob, 2), np.round(thirdclass_survival_prob, 2),
         output['Decisions'].sum(), (output['Decisions'] & output['Reality']).sum(),
         FP, FN, TP, TN]
hash_list = ['4fe6b7a204127acbec520ac2997133dc', 
            '8ee5d21b272d43a875504f3e5845e141',
            'b9c48c2d04160ef1ff72dba569292058',
            '7a7763d4618eb5666515c04dd53a3c08', 
            '5d6182b8169f820c3e247e91131138ea',
            'e2ef524fbf3d9fe611d5a8e90fefdc9c',
            'ad61ab143223efbc24c7d2583be69251',
            '37693cfc748049e45d87b8c7d8b9aacd', 
            '6364d3f0f495b6ab9dcf8d3b5c6e0b01', 
            'ad61ab143223efbc24c7d2583be69251', 
            '013d407166ec4fa56eb1e1f8cbe183b9']
assert all([get_hash(t) == hash_list[i] for (i, t) in enumerate(tests)])
print("All tests passed! You are awesome!!!")

In [ ]:
import matplotlib.image as mpimg
img = mpimg.imread('cute_quokka.jpg')
imgplot = plt.imshow(img)
imgplot.axes.get_xaxis().set_visible(False)
imgplot.axes.get_yaxis().set_visible(False)
plt.show()
print('Congrats! You made it to the end of the lab!!!')